In [2]:
!pip install -q tensorflow-text


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.5 MB/s eta 0:00:00


In [3]:
import tensorflow_hub as hub
import tensorflow_text as text


In [96]:
# preprocess_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
# encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [97]:
# bert_preprocess_model = hub.KerasLayer(preprocess_url)



In [31]:
# text_test = ["I love coding" ,"coding makes me happy" ]


In [99]:
# text_preprocessed = bert_preprocess_model(text_test)
# text_preprocessed. keys()


In [100]:
# bert_model = hub.KerasLayer(encoder_url)


In [101]:
# bert_result=bert_model(text_preprocessed)

In [102]:
# bert_result.keys()

In [103]:
# #"pooled_out" is basically embedding of a whole sentence
# bert_result['pooled_output']

In [104]:
# #"pooled_out" is basically embedding of a single word
# bert_result['sequence_output']

CLASSIFICATION OF SPAM EMAIL USING BERT

In [4]:
import pandas as pd


In [5]:
#Reading the file using pandas
df=pd.read_csv("spam1.csv",encoding = "ISO-8859-1")

In [6]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [33]:
#Dropping the unwanted columns and renaming the features
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],inplace=True)
df.rename(columns = {'v1':'Category', 'v2':'Message'}, inplace = True)
df.head(5)

KeyError: ignored

In [34]:
#applying lambda function to convert "ham" into 0 and spam into 1
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head(5)

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [9]:
df.groupby("spam").describe()


Category                    Message         \
        count unique   top  freq   count unique   
spam                                              
0        4825      1   ham  4825    4825   4516   
1         747      1  spam   747     747    653   

                                                              
                                                    top freq  
spam                                                          
0                                Sorry, I'll call later   30  
1     Please call our customer service representativ...    4

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
 2   spam      5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [35]:
#splitting data into Train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'],stratify=df["spam"])


In [12]:
len(X_test)

1393

In [13]:
len(X_train)

4179

In [14]:
len(y_test)

1393

In [36]:
#Using BERT 
preprocess_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [16]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)



In [32]:
#Trying to figure what actually happened after applying Bert PreProcess and  Encoder
def get_sentence_embeding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']
##"pooled_out" is basically embeding of a whole sentence

In [18]:
get_sentence_embeding([
    "huraah, you won $250000 in lottery",
    "hurry up, call for discount "
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.74335104, -0.74821365, -0.98174584, ..., -0.94034326,
        -0.7121398 ,  0.80605626],
       [-0.8052674 , -0.38586906, -0.78287494, ..., -0.5306579 ,
        -0.69994956,  0.92930776]], dtype=float32)>

In [19]:
import tensorflow as tf


In [20]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
# here we are using Functional not Sequential

preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

L = tf.keras.layers.Dropout (0.3, name="dropout")(outputs['pooled_output'])
L = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(L)

model = tf.keras.Model(inputs=[text_input], outputs=[L])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [21]:
model.compile(optimizer= "Adam",loss="binary_crossentropy",metrics=["accuracy"])


In [22]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 53s 326ms/step - loss: 0.3745 - accuracy: 0.8557
Epoch 2/5
131/131 [==============================] - 45s 340ms/step - loss: 0.2687 - accuracy: 0.8818
Epoch 3/5
131/131 [==============================] - 44s 333ms/step - loss: 0.2217 - accuracy: 0.9045
Epoch 4/5
131/131 [==============================] - 44s 337ms/step - loss: 0.1948 - accuracy: 0.9246
Epoch 5/5
131/131 [==============================] - 44s 335ms/step - loss: 0.1795 - accuracy: 0.9265


In [23]:
model.evaluate(X_test,y_test)

44/44 [==============================] - 15s 331ms/step - loss: 0.1669 - accuracy: 0.9419


[0.16690659523010254, 0.9418520927429199]

In [29]:
#Lests check with examples.
#here first 3 are spam and 4th one is Not spam

example=["Save up to 70% on Life Insurance. Why Spend More Than You Have To?Life Quote Savings Ensuring you",
         "I thought you might like these: 1) Slim Down - Guaranteed to lose 10-12 lbs in 30 days http://www.fr.com",
         "Lowest Rate Services Conferencing Made Easy Only 18 Cents Per Minute! (Including Long Distance!)No s...",
         "your account is going to expire, please visit our official website www.oracle.org"]


In [30]:
model.predict(example)


1/1 [==============================] - 0s 44ms/step


array([[0.3723036 ],
       [0.21522641],
       [0.05573575],
       [0.53784066]], dtype=float32)